<a href="https://colab.research.google.com/github/drfperez/openair/blob/main/Gencat2Openair.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

import pandas as pd
from google.colab import files

# Pujar el fitxer CSV
print("Carrega el teu fitxer CSV:")
uploaded = files.upload()

# Carregar el fitxer CSV
file_name = list(uploaded.keys())[0]
df = pd.read_csv(file_name)

# Comprovar les columnes necessàries
required_columns = ['data', 'contaminant', 'h01', 'h02', 'h03', 'h04', 'h05', 'h06', 'h07', 'h08',
                    'h09', 'h10', 'h11', 'h12', 'h13', 'h14', 'h15', 'h16', 'h17', 'h18', 'h19',
                    'h20', 'h21', 'h22', 'h23', 'h24']
if not all(col in df.columns for col in required_columns):
    raise ValueError("El fitxer no conté les columnes requerides.")

# Expandir les hores en files amb un format de temps
df = df.melt(id_vars=['data', 'contaminant'],
             value_vars=[f"h{i:02}" for i in range(1, 25)],
             var_name='hora',
             value_name='valor')

# Convertir la columna 'hora' en un format d'hora real
df['hora'] = df['hora'].str.extract('(\d+)').astype(int) - 1  # Convertir 'h01', 'h02', ... a 0-23
df['data'] = pd.to_datetime(df['data'], errors='coerce') + pd.to_timedelta(df['hora'], unit='h')

# Eliminar la columna 'hora' i reestructurar les dades
df = df.drop(columns=['hora'])

# Seleccionar només les columnes necessàries (data, contaminant i les hores expandides)
df = df[['data', 'contaminant', 'valor']]

# Pivotar el DataFrame per tenir una columna per cada contaminant
pivot_df = df.pivot(index='data', columns='contaminant', values='valor').reset_index()

# Ordenar per data
pivot_df = pivot_df.sort_values(by='data')

# Convertir les dates a format amb espai (en lloc de T) per ISO abans de guardar el CSV
pivot_df['data'] = pivot_df['data'].dt.strftime('%Y-%m-%d %H:%M:%S')

# Renombrar la columna 'data' com 'date'
pivot_df = pivot_df.rename(columns={'data': 'date'})

# Convertir els noms dels contaminants a minúscules
pivot_df.columns = [col.lower() if isinstance(col, str) and col != 'date' else col for col in pivot_df.columns]

# Substituir els valors buits (NaN) per 'NA'
pivot_df = pivot_df.fillna('NA')

# Guardar el fitxer CSV processat
output_file = 'processed_data.csv'
pivot_df.to_csv(output_file, index=False)

# Descarregar el fitxer processat
files.download(output_file)
print(f"El fitxer processat s'ha desat com: {output_file}")

Carrega el teu fitxer CSV:


Saving martorell281224.csv to martorell281224.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

El fitxer processat s'ha desat com: processed_data.csv
